In [ ]:
library(stringr)
library(XML)
library(maps)
library(httr)
library(janitor)
library(stringi)
url <- "https://en.wikipedia.org/wiki/List_of_World_Heritage_in_Danger"
doc <- htmlParse(rawToChar(GET(url)$content))
dati_formattati_heritage <- readHTMLTable(doc,stringsAsFactors=FALSE) #uguale a prima.
danger_table <- dati_formattati_heritage[[2]]
danger_table <- janitor::row_to_names(danger_table, 1, remove_rows_above = FALSE)
names(danger_table)
colnames(danger_table) <- c("name","locn","crit","yins","yend")
danger_table$Criteria <- ifelse(str_detect(danger_table$Criteria,
"Natural")== TRUE, "nat","cult")
Endangered_clean <- str_extract_all(danger_table$Endangered, "^[[:digit:]]{4}")
danger_table$Endangered <- Endangered_clean
Endangered_clean <- as.numeric(stri_extract_last_regex(danger_table$Endangered, "\\d{4}"))
#Impostazioni per l’estrazione, sono state ottenute osservando la scrittura della
#variabile location
reg_y <- "[/][ -]*[[:digit:]]*[.]*[[:digit:]]*[;]"
reg_x <- "[;][ -]*[[:digit:]]*[.]*[[:digit:]]*"
#Prima coordinata e creazione di nuova variabile
y_coords <- str_extract(danger_table$Location,reg_y)
y_coords <- as.numeric(str_sub(y_coords, 3, -2))
danger_table$y_coords <- round(y_coords,2) #approssimazione alla seconda cifra decimale
#Seconda coordinata e creazione di nuova variabile
x_coords <- str_extract(danger_table$Location,reg_x)
x_coords <- as.numeric(str_sub(x_coords, 3, -1))
danger_table$x_coords <- round(x_coords,2) #approssimazione alla seconda cifra decimale
#Cancellazione della vecchia variabile
danger_table$Location <- NULL
pch <- ifelse(danger_table$Criteria =="nat",19,2)
map("world", col="darkgrey", lwd= 0.5, mar=c(0.1,0.1,0.1,0.1))
points(danger_table$x_coords,danger_table$y_coords,pch=pch)
box()